<a href="https://colab.research.google.com/github/rgclapp007/gp211-class-notebooks/blob/main/regularization/miss1d.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!python -m pip install "sep_plot @ git+http://zapad.stanford.edu/bob/pySepPlot.git@2bffacb9fb36963339a0834c2b04a0aedff91db4"


In [ ]:
import numba
from genericSolver.pyOperator import Operator
from sep_python.sep_vector import FloatVector,get_sep_vector
import numpy as np
class tconv(Operator):

  def __init__(self,mod:FloatVector,dat:FloatVector,filt:np.ndarray):
    if not isinstance(mod,FloatVector) or not isinstance(dat,FloatVector):
      raise Exception(f"Expecting float vectors got {type(mod)} and {type(dat)}")
    nm=mod.get_hyper().get_ns()
    nd=dat.get_hyper().get_ns()

    if not isinstance(filt,np.ndarray):
      raise Exception("Expecting filter to be an n-d array")

    if len(list(filt.shape))!=1:
      raise Exception("Expecting filter to be 1-D")
    
    if len(nm) !=1 or len(nd)!=1:
      raise Exception("Expecting 1-D vectors")
    
    if nd[0]!=nm[0]+filt.shape[0]-1:
      raise Exception("Expecting size of data to be len(filt)+len(mod)-1")
    
    self._filt=np.copy(filt)

    super().__init__(mod,dat)

  def forward(self,add,mod,dat):
    self.checkDomainRange(mod,dat)
    if not add:
      dat.zero()

    tconv_forward(mod.get_nd_array(),dat.get_nd_array(),self._filt)

  def adjoint(self,add,mod,dat):
    self.checkDomainRange(mod,dat)
    if not add:
      mod.zero()

    tconv_adjoint(mod.get_nd_array(),dat.get_nd_array(),self._filt)
  
@numba.njit()
def tconv_forward(mod,dat,filt):
  for imod in range(mod.shape[0]):
    for ifilt in range(filt.shape[0]):
      dat[imod+ifilt]+=filt[ifilt]*mod[imod]

@numba.njit()
def tconv_adjoint(mod,dat,filt):
  for imod in range(mod.shape[0]):
    for ifilt in range(filt.shape[0]):
      mod[imod]+=filt[ifilt]*dat[imod+ifilt]


In [ ]:
from genericSolver.pyProblem import ProblemL2Linear
from genericSolver.pyLinearSolver import LCGsolver
from genericSolver.pyStopper import BasicStopper 
import holoviews as hv
hv.extension('bokeh','matplotlib')
from sep_plot import Dots

def estimate_1d(inp,filt):
  model=inp.clone()
  nm=model.get_hyper().get_ns()
  data=get_sep_vector(ns=[nm[0]+filt.shape[0]-1])
  unknown=model.clone()
  un=unknown.get_nd_array()
  un[:]=1
  for ival,val in enumerate(model.get_nd_array()):
    if val !=0:
      un[ival]=0
  
  op=tconv(model,data,filt)
  prob=ProblemL2Linear(model,data,op,grad_mask=unknown)
  stop=BasicStopper(niter=nm[0])
  solve=LCGsolver(stop)
  solve.run(prob)
  return Dots(inp)+Dots(model)+Dots(filt)







In [ ]:
import holoviews as hv
hv.extension('bokeh','matplotlib')
inp=get_sep_vector(np.array([0.,0.,0.,0.,.5,0.,1.,.5,1.,0.,0.,0.,0.,0.,0.],dtype=np.float32))
filt=np.array([1,-1.])
est=estimate_1d(inp,filt)
est

In [ ]:
%xmode verbose

import holoviews as hv
hv.extension('bokeh','matplotlib')
filt=np.array([1.,-2.,1.])
print(type(inp),type(filt))
estimate_1d(inp,filt)

In [ ]:
import holoviews as hv
hv.extension('bokeh','matplotlib')
filt=np.array([1.,1.])
print(type(inp),type(filt))
estimate_1d(inp,filt)

In [ ]:
import holoviews as hv
hv.extension('bokeh','matplotlib')
filt=np.array([1.,2.,3.])
print(type(inp),type(filt))
estimate_1d(inp,filt)

In [ ]:
import holoviews as hv
hv.extension('bokeh','matplotlib')
filt=np.array([3.,2.,1.])
print(type(inp),type(filt))
estimate_1d(inp,filt)

In [ ]:
def estimate_model(inp,filt):
  model=inp.clone()
  nm=model.get_hyper().get_ns()
  data=get_sep_vector(ns=[nm[0]+filt.shape[0]-1])
  unknown=model.clone()
  un=unknown.get_nd_array()
  un[:]=1
  for ival,val in enumerate(model.get_nd_array()):
    if val !=0:
      un[ival]=0
  
  op=tconv(model,data,filt)
  prob=ProblemL2Linear(model,data,op,grad_mask=unknown)
  stop=BasicStopper(niter=nm[0])
  solve=LCGsolver(stop)
  solve.setDefaults(save_obj=True,save_res=True,save_model=True)
  solve.run(prob)
  return prob,solve

import holoviews as hv
hv.extension('bokeh','matplotlib')
inp=get_sep_vector(np.array([0.,0.,0.,0,0,0,0.,0.,.5,0.,1.,.5,1.,0.,0.,0.,0.,0.,0.,0,0,0,0,0],dtype=np.float32))

filt=np.array([1.,-1.])
print(type(inp),type(filt))
prob,solver=estimate_model(inp,filt)

In [ ]:
import holoviews as hv
hv.extension('bokeh','matplotlib')
Dots(solver.model[1])+Dots(solver.model[3])+Dots(solver.model[7] )

In [ ]:
import holoviews as hv
hv.extension('bokeh','matplotlib')
Dots(solver.model[11])+Dots(solver.model[15])+Dots(solver.model[17] )

In [ ]:
print(solver.obj)